# Traitlets Tutorial
This tutorial shows how the Python package `traitlets` is used by PODPAC developers to make nodes. The same style should be used by new node developers.

This tutorial assume knowledge of Python (syntax, basic types, classes, try-except blocks, and decorators)

In [1]:
import podpac
import traitlets as tl  # We use the shortform 'tl'


# A short, full-featured example
## Defining a PODPAC Node using Traitlets

In [2]:
# Create a new custom PODPAC Node
class MyNode(podpac.Node):
    # Define an input that's another specific type of Node
    input = tl.Instance(podpac.algorithm.CoordData)
    
    # Define attributes that affect the output (note these are 'tagged' with attr=True)
    dimension = tl.Unicode(allow_none=True, default_value=None).tag(attr=True)  # The default for this trait is specified
    scale = tl.Float().tag(attr=True)
    
    # Define internal class attributes (note, these are not 'tagged')
    _other_attr = tl.Float(default_value=-1)  # The default is -1 for this trait
        
    # Define a default for this trait based on a function
    # Note: This function is only called when the 'scale' attribute is first requested, and only if it hasn't been specified
    @tl.default('scale')  # This is a Python 'decorator', it wraps the next function (_default_scale) and adds functionality to it
    def _default_scale(self):
        return self._other_attr 
    
    @tl.default('input')
    def _default_input(self):
        return podpac.algorithm.CoordData()
    
    # Set up an observer for the 'dimension' attribute using traits
    # Note: This function is called whenever the 'dimension' attribute is changed
    @tl.observe('dimension')
    def _modify_coord_data_dim(self, change):
        print ("Changing extracted dimension from %s to %s" % (change['old'], change['new']))
        self.input.coord_name = change['new']  # Change coord_name attribute of input node
        
    # Define the eval method, which is not implemented as part of the base PODPAC node
    def eval(self, coordinates, output=None, method=None):
        o = self.input.eval(coordinates)
        return o*scale
    
    # Define a function to print all of the class traits -- this is for nice display for this tutorial and not necessary for the node to function
    def __repr__(self):
        return self.json_pretty
# input = podpac.algorithm.CoordData()
my_node = MyNode()
print ("Note that tagged attributes are automatically included in the JSON representation of the Node.")
my_node

Note that tagged attributes are automatically included in the JSON representation of the Node.


{
    "nodes": {
        "MyNode": {
            "plugin": "__main__",
            "node": "MyNode",
            "attrs": {
                "dimension": null,
                "scale": -1.0
            }
        }
    }
}

## Setting Defaults
Notes, this method for handling defaults can be used instead of custom contructors or initializers for nodes

In [3]:
# Demonstrate when defaults are called
m1 = MyNode()

print("This line causes the default for scale to be computed based on the value of _other_attr")
print ("m1._other_attr:", m1._other_attr, '\tm1.scale:', m1.scale)  
print("Changing _other_attr will not change m1.scale")
m1._other_attr = -2  
print ("m1._other_attr (changed):", m1._other_attr, '\tm1.scale:', m1.scale)

print("If we change _other_attr before asking for scale, the default for scale will change")
m2 = MyNode()
m2._other_attr = -2
print ("m2._other_attr:", m2._other_attr, '\tm2.scale:', m2.scale)

print("If we set scale at instantiation of the class, the value of _other_attr will not affect scale")
m3 = MyNode(scale=7)
m3._other_attr = -3
print ("m3._other_attr:", m3._other_attr, '\tm3.scale:', m3.scale)

print("Same goes for instantiation of both when the class is created")
m4 = MyNode(_other_attr=-4, scale=7)
print ("m4._other_attr:", m4._other_attr, '\tm4.scale:', m4.scale)

This line causes the default for scale to be computed based on the value of _other_attr
m1._other_attr: -1.0 	m1.scale: -1.0
Changing _other_attr will not change m1.scale
m1._other_attr (changed): -2.0 	m1.scale: -1.0
If we change _other_attr before asking for scale, the default for scale will change
m2._other_attr: -2.0 	m2.scale: -2.0
If we set scale at instantiation of the class, the value of _other_attr will not affect scale
m3._other_attr: -3.0 	m3.scale: 7.0
Same goes for instantiation of both when the class is created
m4._other_attr: -4.0 	m4.scale: 7.0


## Demonstration of Traitlets Observations
This shows how a function is called whenever an 'observed' attribute is modified.

In [4]:
my_node = MyNode()
print ("input.coord_name:", my_node.input.coord_name)
my_node

input.coord_name: 


{
    "nodes": {
        "MyNode": {
            "plugin": "__main__",
            "node": "MyNode",
            "attrs": {
                "dimension": null,
                "scale": -1.0
            }
        }
    }
}

In [5]:
my_node.dimension = 'lon'
print ("input.coord_name:", my_node.input.coord_name)
my_node

Changing extracted dimension from None to lon
input.coord_name: lon


{
    "nodes": {
        "MyNode": {
            "plugin": "__main__",
            "node": "MyNode",
            "attrs": {
                "dimension": "lon",
                "scale": -1.0
            }
        }
    }
}

In [6]:
# Note, the observe function is also called on initialization
my_node = MyNode(dimension='lat')
print ("input.coord_name:", my_node.input.coord_name)
my_node

Changing extracted dimension from None to lat
input.coord_name: lat


{
    "nodes": {
        "MyNode": {
            "plugin": "__main__",
            "node": "MyNode",
            "attrs": {
                "dimension": "lat",
                "scale": -1.0
            }
        }
    }
}

# Trailets Basics
Traitlets is a pure Python library used to add `traits` or `typing` back into Python. Python is inherently a `type-less` language, and relies or so-called `duck-typing` to determine the type of a variable. For example, Python does not discriminate between a string and an integer or a float. Traitlets allows Python programmers to ensure that their class attributes have the correct type. 

Among `traitlets` features, PODPAC uses the:
* Typing of class attributes
    * This includes simple typing (Int, Float, Unicode, Dict, etc.)
    * But also more complex typing (correct instance of a class, with the correct dimensions, etc.).
* Ability to set defaults for attributes
* Tagging of class attributes
* Observing a class attribute for changes 

## Creating a Traited Class

In [7]:
# Define the class
class MyClass(tl.HasTraits):
    floating_point = tl.Float()
    integer = tl.Int()
    string = tl.Unicode()

    # Shortcut function used to print values of the class
    def __repr__(self):
        print_str = []
        for name in self.trait_names():
            print_str.append(name + ': ' + str(getattr(self, name)))
        return '\n'.join(print_str)
    
# Create an instance of the class
my_class = MyClass()

In [8]:
# Traits have defaults
my_class

floating_point: 0.0
integer: 0
string: 

In [9]:
# We can assign values to the traits that match their type
my_class.integer = 1
my_class.floating_point = 2.1
my_class.string = 'abc'
my_class

floating_point: 2.1
integer: 1
string: abc

In [10]:
# but if the type doesn't match, an exception is thrown
try: 
    my_class.floating_point = '1'
except tl.TraitError as e:
    print(e)
try: 
    my_class.integer = 1.1
except tl.TraitError as e:
    print(e)
try: 
    my_class.string = 8.67
except tl.TraitError as e:
    print(e)

The 'floating_point' trait of a MyClass instance must be a float, but a value of '1' <class 'str'> was specified.
The 'integer' trait of a MyClass instance must be an int, but a value of 1.1 <class 'float'> was specified.
The 'string' trait of a MyClass instance must be a unicode string, but a value of 8.67 <class 'float'> was specified.


**Note**: The integer trait threw an exception when we provided a floating point number. 

If you just want to round the integer, you can use the Cast-`<Trait>` or `C<Trait>` version of a trait.

In [11]:
# Define a new class that inherits from the previous class
class MyClass2(MyClass):
    cast_integer = tl.CInt()  # Create a trait with a cast integer version
    string = tl.CUnicode()    # Over-write the unicode trait in the base class with the cast version
my_class2 = MyClass2()
my_class2

cast_integer: 0
floating_point: 0.0
integer: 0
string: 

In [12]:
my_class2.cast_integer = 1.6  # this is cast to 1
my_class2.string = 8.76  # This is cast to a string
my_class2

cast_integer: 1
floating_point: 0.0
integer: 0
string: 8.76

### Traits of type "Class Instance"

In [13]:
class MyClass3(MyClass):
    my_class2 = tl.Instance(MyClass2)
    my_class = tl.Instance(MyClass)
my_class3 = MyClass3()
my_class3.my_class2 = MyClass2()
my_class3.my_class = MyClass2()

In [14]:
my_class3.my_class = MyClass2()  # Can assign child classes (still an instance of MyClass)
try: 
    my_class3.my_class2 = MyClass()  # Cannot assign parent classes (MyClass is not an instance of MyClass2)
except tl.TraitError as e:
    print(e)

The 'my_class2' trait of a MyClass3 instance must be a MyClass2, but a value of class '__main__.MyClass' (i.e. floating_point: 0.0
integer: 0
string: ) was specified.


## Initializing Traits

Traits provides three mechanisms for initializing the values of traited attributes that differ from the normal Python approach:
* Setting a value when instantiating the class
    * Normally this would require some logic in the `__init__` function of the class (i.e. the class constructor)
* Setting a default value when defining the trait
* Setting a default value through a function the first time an attribute is requested

**NOTE: GOTCHA**
The default values are not ALWAYS used, and they are lazy -- that is they are only set upon request. So, if the default depends on another attribute, and the other attribute changes, the value of the default depends on when it was first requested. An example will be shown below. 


In [15]:
class SetValues(tl.HasTraits):
    no_default = tl.Unicode()  # Automatic default value of ''
    simple_default = tl.Unicode(default_value='simple')  # Default value set while defining the trait
    computed_default = tl.Unicode()  # function for this attribute's default is defined next
    
    @tl.default('computed_default')  # Using the traitlets default decorator
    def _computed_default(self):
        return ('"' + self.no_default + '" -- "' + self.simple_default + '"').upper()
    
    # Shortcut function used to print values of the class
    def __repr__(self):
        print_str = []
        for name in self.trait_names():
            print_str.append(name + ': ' + str(getattr(self, name)))
        return '\n'.join(print_str)

In [16]:
# Only default values
SetValues()

computed_default: "" -- "SIMPLE"
no_default: 
simple_default: simple

In [17]:
# Set values when creating class
SetValues(computed_default='computed_default_set', no_default='no_default_set')

computed_default: computed_default_set
no_default: no_default_set
simple_default: simple

In [18]:
# Gotcha for computed value. It is only computed after being asked for, not when object is instantiated
sv1 = SetValues()
sv2 = SetValues()
sv1.no_default = "computed_default calculated after changing attribute"
sv2.computed_default  # Ask for the computed_default so that it is computed and set
sv2.no_default = "computed_default already calculated"  # change no_default's value

print('sv1', sv1)
print('sv2', sv2)

sv1 computed_default: "COMPUTED_DEFAULT CALCULATED AFTER CHANGING ATTRIBUTE" -- "SIMPLE"
no_default: computed_default calculated after changing attribute
simple_default: simple
sv2 computed_default: "" -- "SIMPLE"
no_default: computed_default already calculated
simple_default: simple


## Tagging Attributes
Traits allows users to put arbitrary metadata on traited attributes. PODPAC uses the 'attr' metadata to determine if an attribute should be part of a pipeline definition. 


In [19]:
class Tags(tl.HasTraits):
    tagged = tl.Unicode().tag(attr=True, arbitrary_tag="arbitrary")
    untagged = tl.Unicode()
tags = Tags()
print ("tagged.attr", tags.trait_metadata('tagged', 'attr'))
print ("tagged.arbitrary_tag", tags.trait_metadata('tagged', 'attr'))
print ("untagged.attr", tags.trait_metadata('untagged', 'attr'))
print ("untagged.arbitrary_tag", tags.trait_metadata('untagged', 'attr'))
print ("Available metadata for tagged", tags.traits()['tagged'].metadata)
print ("Available metadata for untagged", tags.traits()['tagged'].metadata)

tagged.attr True
tagged.arbitrary_tag True
untagged.attr None
untagged.arbitrary_tag None
Available metadata for tagged {'attr': True, 'arbitrary_tag': 'arbitrary'}
Available metadata for untagged {'attr': True, 'arbitrary_tag': 'arbitrary'}


## Observing Traits
Traits implements the observer pattern. Any traited attribute can be observed for a change, triggering a function call. 

In [20]:
class Observe(tl.HasTraits):
    watch_me = tl.Any()
    
    @tl.observe('watch_me')
    def __watch_me_changed(self, change):
        print (change)

print ('Observed functions are called when the function is initialized:')
obs = Observe(watch_me='old')
print ('Observed function are called whenever the value of the attribute changes')
obs.watch_me = 'new'
obs.watch_me = 'new again'
obs.watch_me = 42
print ('Observed function are not called whenever the value of the attribute stays the same')
obs.watch_me = 42

Observed functions are called when the function is initialized:
{'name': 'watch_me', 'old': None, 'new': 'old', 'owner': <__main__.Observe object at 0x000002BE6870A5F8>, 'type': 'change'}
Observed function are called whenever the value of the attribute changes
{'name': 'watch_me', 'old': 'old', 'new': 'new', 'owner': <__main__.Observe object at 0x000002BE6870A5F8>, 'type': 'change'}
{'name': 'watch_me', 'old': 'new', 'new': 'new again', 'owner': <__main__.Observe object at 0x000002BE6870A5F8>, 'type': 'change'}
{'name': 'watch_me', 'old': 'new again', 'new': 42, 'owner': <__main__.Observe object at 0x000002BE6870A5F8>, 'type': 'change'}
Observed function are not called whenever the value of the attribute stays the same


# Where to go from here
For more detailed help related to traitlets, look at the [Traitlets Documentation](https://traitlets.readthedocs.io/en/stable/).

For more detailed help on PODPAC, look at the notebook examples in the repository (where this notebook lives) or look at the [PODPAC Documentation](https://creare-com.github.io/podpac-docs/).